In [2]:
!pip install nltk==3.5
!pip install gensim
!pip install umap-learn
!pip install wikipedia
!pip install unidecode

In [3]:
import re
import nltk
from nltk.util import ngrams
from nltk.corpus import stopwords
import wikipedia
import string
from unidecode import unidecode
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import urllib.request
import bz2
import gensim
import warnings
import numpy as np
from gensim.models import word2vec
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings('ignore')
nltk.download('punkt')

import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Definição do Corpus

## Base

In [4]:
wikipedia.set_lang("pt")
bh = wikipedia.page("Belo Horizonte")

In [5]:
corpus = bh.content

In [6]:
print(corpus)

Belo Horizonte é um município brasileiro e a capital do estado de Minas Gerais. Sua população é de 2 315 560 habitantes, segundo o censo de 2022, sendo o sexto município mais populoso do país, o terceiro da Região Sudeste e o primeiro de seu estado. Com uma área de aproximadamente 331 km², possui uma geografia diversificada, com morros e baixadas. Com uma distância de 716 quilômetros de Brasília, é a segunda capital de estado mais próxima da capital federal, depois de Goiânia.
Cercada pela Serra do Curral, que lhe serve de moldura natural e referência histórica, foi planejada e construída para ser a capital política e administrativa do estado mineiro sob influência das ideias do positivismo, num momento de forte apelo da ideologia republicana no país. Sofreu um inesperado crescimento populacional acelerado, chegando a mais de um milhão de habitantes com quase setenta anos de fundação. Entre as décadas de 1930 e 1940, ocorreu também o avanço da industrialização, além de muitas construçõ

Selecionamos algumas frases do corpus de BH da wikipedia.

Conside a lista abaixo como nosso corpus de documentos. Cada elemento da lista, considere como um único documento.

In [7]:
documentos = \
["Belo Horizonte é um município brasileiro e a capital do estado de Minas Gerais",
"A populacao de Belo Horizonte é estimada em 2 501 576 habitantes, conforme estimativas do Instituto Brasileiro de Geografia e Estatística",
"Belo Horizonte já foi indicada pelo Population Crisis Commitee, da ONU, como a metrópole com melhor qualidade de vida na América Latina",
"Belo Horizonte é mundialmente conhecida e exerce significativa influência nacional e até internacional, seja do ponto de vista cultural, econômico ou político",
"Belo Horizonte é a capital do segundo estado mais populoso do Brasil, Minas Gerais"]

## Preprocessamento

<b> Atividade </b>

1) Escreva uma método que realiza o pré-processamento da lista de <b>documentos</b>.

O método deve, para cada documento:
- tokenizar cada palavra
- remover stopwords
- remover números
- remover pontuções
- remover acentos

In [8]:
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
nltk.word_tokenize(corpus, language='portuguese')
portugues_stops = stopwords.words('portuguese')

In [10]:
def pre_processamento(txt):
  tokens = nltk.word_tokenize(txt, language='portuguese')
  tokens = [t.lower() for t in tokens]
  tokens = [t for t in tokens if t not in portugues_stops]
  tokens = [re.sub('\d', '', t) for t in tokens]
  tokens = [t for t in tokens if t not in string.punctuation]
  tokens = [unidecode(t) for t in tokens]
  return tokens

In [11]:
documentos[1]

'A populacao de Belo Horizonte é estimada em 2 501 576 habitantes, conforme estimativas do Instituto Brasileiro de Geografia e Estatística'

In [12]:
pre_processamento(documentos[1])

['populacao',
 'belo',
 'horizonte',
 'estimada',
 'habitantes',
 'conforme',
 'estimativas',
 'instituto',
 'brasileiro',
 'geografia',
 'estatistica']

In [13]:
doc_proc = [' '.join(pre_processamento(doc)) for doc in documentos]

In [14]:
doc_proc

['belo horizonte municipio brasileiro capital estado minas gerais',
 'populacao belo horizonte estimada habitantes conforme estimativas instituto brasileiro geografia estatistica',
 'belo horizonte indicada population crisis commitee onu metropole melhor qualidade vida america latina',
 'belo horizonte mundialmente conhecida exerce significativa influencia nacional internacional ponto vista cultural economico politico',
 'belo horizonte capital segundo estado populoso brasil minas gerais']

# Representação Textual

## TD-IDF

Dica de leitura: https://kavita-ganesan.com/tfidftransformer-tfidfvectorizer-usage-differences/#.XklQxnVKj7c

Para representar o texto com TF-IDF utilizamos o TfidfVectorizer. A seguir apresentamos instruções sobre como utilizá-lo.

```python
#primeiro criamos o objeto
vect = TfidfVectorizer()
vect #aqui você pode observa todos os parâmetros que o objeto possui
## Existem alguns parâmetros, opcionais, que podemos informar para uma possível melhora do nosso modelo. Por exemplo:
### inclui 1-grams e 2-grams
vect.set_params(ngram_range=(1, 2))
### ignora termos que a aparecem em mais de 50% dos documentoss
vect.set_params(max_df=0.5)
### só considero termos que aparecem em ao menos 2 documentos
vect.set_params(min_df=2)

#depois aplicamos fit_transform para transformar o texto em números
docs_tdidf = vect.fit_transform(docs)

#o docs_tdidf é uma matriz com os números que representam cada um dos textos.
## Conseguimos verificar a dimensão desta matriz:
print(docs_tdidf.shape)

#Para visualizar as features capturadas pelo TF-IDF utilize:
print(vect.get_feature_names_out())
#Para visualizar os vetores correspondentes a cada palavara utilize:
df = pd.DataFrame(docs_tdidf.T.todense(), index=vect.get_feature_names_out(), columns=["doc"+str(i+1) for i in range(0,len(docs))])
df
```

<b> Atividade: </b>

2) Faça o TDIFTVectorizer nos documentos da variável <b>documentos</b> sem alterar nenhum parâmetro.

In [15]:
vect_tfidf = TfidfVectorizer()
docs_tdidf = vect_tfidf.fit_transform(doc_proc)

In [16]:
docs_tdidf

<5x42 sparse matrix of type '<class 'numpy.float64'>'
	with 55 stored elements in Compressed Sparse Row format>

<b> Atividade </b>

3) Imprima o shape do resultado da atividade 2

In [17]:
docs_tdidf.shape

(5, 42)

<b> Atividade </b>

4) Imprima as features capturadas em 2.

In [18]:
vect_tfidf.get_feature_names_out()

array(['america', 'belo', 'brasil', 'brasileiro', 'capital', 'commitee',
       'conforme', 'conhecida', 'crisis', 'cultural', 'economico',
       'estado', 'estatistica', 'estimada', 'estimativas', 'exerce',
       'geografia', 'gerais', 'habitantes', 'horizonte', 'indicada',
       'influencia', 'instituto', 'internacional', 'latina', 'melhor',
       'metropole', 'minas', 'mundialmente', 'municipio', 'nacional',
       'onu', 'politico', 'ponto', 'populacao', 'population', 'populoso',
       'qualidade', 'segundo', 'significativa', 'vida', 'vista'],
      dtype=object)

5) Imprima os vetores correspondentes a cada palavra de cada documento.

In [19]:
df = pd.DataFrame(docs_tdidf.T.todense(),
                  index=vect_tfidf.get_feature_names_out(),
                  columns=["doc"+str(i+1) for i in range(0,len(documentos))])
df

,doc1,doc2,doc3,doc4,doc5
america,0.000000,0.000000,0.295474,0.000000,0.000000
belo,0.219592,0.157916,0.140795,0.135024,0.193603
brasil,0.000000,0.000000,0.000000,0.000000,0.406297
brasileiro,0.371802,0.267376,0.000000,0.000000,0.000000
capital,0.371802,0.000000,0.000000,0.000000,0.327798
commitee,0.000000,0.000000,0.295474,0.000000,0.000000
conforme,0.000000,0.331405,0.000000,0.000000,0.000000
conhecida,0.000000,0.000000,0.000000,0.283363,0.000000
crisis,0.000000,0.000000,0.295474,0.000000,0.000000
cultural,0.000000,0.000000,0.000000,0.283363,0.000000


## Bag of Words

Para representar o bag of words utilizamos o CountVectorizer

```python
#primeiro criamos o objeto
vect_bag = CountVectorizer(binary=False) #se binary = False -> ocorre a contagem da frequência em que a palavra aparece
vect_bag #imprime os parâmetros

```

<b> Atividade </b>

6) Faça o CountVectorizer nos documentos da variável <b>documentos</b> considerando binary = True

In [20]:
vect_bag = CountVectorizer(binary=True)
vect_bag

CountVectorizer(binary=True)

<b> Atividade </b>

7) Imprima o índice correspondente a cada token da lista retornada por vect_bag.get_feature_names_out()

In [21]:
docs_bag = vect_bag.fit_transform(doc_proc)
docs_bag

<5x42 sparse matrix of type '<class 'numpy.int64'>'
	with 55 stored elements in Compressed Sparse Row format>

In [22]:
df = pd.DataFrame(docs_bag.T.todense(),
                  index=vect_bag.get_feature_names_out(),
                  columns=["doc"+str(i+1) for i in range(0,len(documentos))])
df

,doc1,doc2,doc3,doc4,doc5
america,0,0,1,0,0
belo,1,1,1,1,1
brasil,0,0,0,0,1
brasileiro,1,1,0,0,0
capital,1,0,0,0,1
commitee,0,0,1,0,0
conforme,0,1,0,0,0
conhecida,0,0,0,1,0
crisis,0,0,1,0,0
cultural,0,0,0,1,0


## Embedding

### Treinando seu embedding

Aqui vamos utilizar o corpus machado. São textos/contos escritos por Machado de Assis.
Esse corpus é diponibilizado pelo NLTK.

In [23]:
from nltk.corpus import machado

In [24]:
import nltk
from nltk.corpus import machado
nltk.download('machado')

[nltk_data] Downloading package machado to /root/nltk_data...
[nltk_data]   Package machado is already up-to-date!


True

In [25]:
raw_casmurro = machado.raw('contos/macn001.txt')

O método ''machado_sents()'' retorna todo o texto quebrado pelas setenças e já tokenizado.

As sentenças são separadas pelo "\n". Dentro de cada sentença, divide os tokens separadas pelo espaço.

In [26]:
machado_sents = machado.sents()
print(machado_sents)

[['Conto', ',', 'Contos', 'Fluminenses', ',', '1870'], ['Contos', 'Fluminenses'], ...]


In [27]:
len(machado_sents)

200559

Vamos relizar um pré-processamento mínimo nos dados. Lembrando que: o pré-processamento é impotatíssimo no resultado final.

<b> Atividade </b>

8) Aplique as técnicas abaixo no documento <b> machado_sents</b>:

- lower
- remoção pontuações

In [28]:
text_preproc = [[word.lower() for word in sentence if word not in string.punctuation ] for sentence in machado_sents]

<b> Treinando o embedding </b>

Para treinar os embeddings existem alguns parâmetros, vide exemplo abaixo:

```python
#Alguns parâmetros:
## size -> dimensão vetor
## min_count -> ignora todas palavras cuja frequência mínima é menor que este
## workers -> quantas threads serão utilizadas para treinar o modelo
## seed -> seed para geração do numero aleatório.
## sg -> 1 para skip-gram; caso contrário CBOW.
## window -> contexto, Distância máxima entre a palavra atual e a prevista em uma frase. O default é 5.
model = word2vec.Word2Vec(text_preproc, min_count=10, workers=4, seed=123, sg=1, vector_size=300, window=5)
```

<b> Atividade </b>

9) Gere os embeddings com o texto processado do documento de Machado de Assis.


In [29]:
model = word2vec.Word2Vec(text_preproc, min_count=10, workers=4, seed=123, sg=1, vector_size=300, window=5)


<b> Atividade </b>

10) Faça os itens abaixo:

- Verifique o vetor de embeddings da variável "dom"
- Verifique a similaridade entre "mulher" e "homem"
- Verifique a similaridade entre "dom" e "casmurro"

In [30]:
model.wv['dom']

array([-4.82467338e-02,  4.37534936e-02,  4.48568026e-03, -1.57960802e-01,
       -2.34140962e-01,  1.49526792e-02,  3.47729996e-02, -5.58436662e-03,
       -1.23906061e-01, -2.00904340e-01, -4.82616872e-02, -9.99530125e-03,
        3.04358266e-02,  6.63698390e-02, -4.40624505e-02,  1.21702857e-01,
        3.57329659e-03,  9.92494226e-02, -4.45967130e-02,  1.71457991e-01,
        1.66280329e-01, -9.03795809e-02,  2.65082042e-03,  1.06708221e-01,
       -2.29413677e-02, -1.98947228e-02,  1.61209434e-01, -4.62887213e-02,
        7.97682628e-03, -2.54930437e-01,  1.78253336e-03, -1.27778333e-02,
       -9.27421525e-02, -5.59959821e-02, -6.40105829e-02, -6.32160995e-03,
       -6.45273924e-02, -9.71542969e-02,  1.92128822e-01, -2.49511935e-02,
       -6.42958060e-02,  1.52001739e-01, -1.98749900e-01, -1.36778757e-01,
       -8.51368159e-03,  5.17838784e-02, -2.23761156e-01,  9.24520791e-02,
        3.08561176e-01,  4.27991664e-03,  1.13252372e-01,  1.56633899e-01,
       -2.03594044e-02, -

In [31]:
model.wv.similarity('mulher', 'homem')

0.40978053

In [32]:
model.wv.similarity('dom', 'casmurro')

0.610038

In [36]:
model.wv.most_similar(positive=['mulher', 'rei'], negative=['homem'], topn=10)

[('príncipe', 0.5507043600082397),
 ('alteza', 0.5437556505203247),
 ('princesa', 0.5396664142608643),
 ('avó', 0.5374821424484253),
 ('cunhada', 0.5359086990356445),
 ('rainha', 0.531740128993988),
 ('el', 0.5280194282531738),
 ('pia', 0.5214385390281677),
 ('incógnita', 0.5198127031326294),
 ('conceição', 0.5196643471717834)]

In [34]:
model.wv.most_similar(positive=['pai', 'avô'], negative=['mãe'], topn=10)

[('querido', 0.7171346545219421),
 ('padrinho', 0.6904242634773254),
 ('talentoso', 0.687262237071991),
 ('patrício', 0.6861327290534973),
 ('protegido', 0.6799179911613464),
 ('colega', 0.6720080375671387),
 ('mr', 0.6704517006874084),
 ('redator', 0.6697953939437866),
 ('mexicano', 0.6697095036506653),
 ('abel', 0.6566097140312195)]

In [37]:
model.wv.most_similar('pai')

[('sobrinho', 0.7329709529876709),
 ('irmão', 0.7308077216148376),
 ('marido', 0.7258265018463135),
 ('tio', 0.7248002886772156),
 ('sogro', 0.7190900444984436),
 ('cunhado', 0.7160969972610474),
 ('padrinho', 0.7094628214836121),
 ('marcos', 0.6682252287864685),
 ('noivo', 0.6659682393074036),
 ('filho', 0.6650660634040833)]

<b> Atividade </b>

Dada as seguintes palavras:

foi, relógio, amor, raiva, brasil.

11) Escreva um método que retorne uma lista com as 5 palavras similares de cada uma das listadas anteriormente.
Imprima a lista das palavras similares, incluindo a palavra origem.

In [38]:
def get_most_similar(modelo, word, max_n):
    words_similares = modelo.wv.most_similar(word, topn=max_n)
    words = [w[0] for w in words_similares]
    words.append(word)
    return words

In [39]:
words_seed = ["foi", "relógio", "amor", "raiva", "brasil"]

In [40]:
for w in words_seed:
    print(get_most_similar(model, w, 5))

['fui', 'correu', 'veio', 'levara', 'seguiu', 'foi']
['quintal', 'embrulho', 'sapato', 'mostrador', 'fumando', 'relógio']
['ciúme', 'orgulho', 'afeto', 'casto', 'egoísmo', 'amor']
['mordia', 'amargura', 'brandamente', 'involuntariamente', 'impaciência', 'raiva']
['parlamento', 'maranhão', 'júri', 'sul', 'município', 'brasil']


### Visualização

Para a visualização dos embeddings iremos  construir um array com todas as palavras retornadas anteriormente.

<b> Atividade </b>

12) Primeiro, gere uma única lista com todas as palavras retornadas anteriomente. O array deve ter tamanho 30.

In [ ]:
all_words = []
for w in words_seed:
    all_words.extend(get_most_similar(model, w, 5))

13) Gere um array com todos os embeddings das palavras anteriores. Este terá dimensão (30,300)

In [ ]:
array_embeddings = np.empty((0,300), dtype='f')
for w in all_words:
    array_embeddings = np.append(array_embeddings, np.array([model.wv[w]]), axis=0)
print(array_embeddings.shape)

<b>Dica: </b> Use a função abaixo para plotar o array 2D que será gerado com o método PCA e TSNE

In [ ]:
def plot_embedding_2d(array_2d, all_words, words_seed):
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    for (x, y), w in zip(array_2d, all_words):
        ax.scatter(x, y, c='red' if w in words_seed else 'blue')
        ax.annotate(w,
                     xy=(x, y),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')

#### PCA

<b> É uma ténica que existe a mais de século. É rápido, determinístico e linear. Essa linearidade limita sua utilidade em domínios complexos, como linguagem natural ou imagens, onde a estrutura não linear. </b>

Mais informações: https://medium.com/towards-artificial-intelligence/machine-learning-dimensionality-reduction-via-principal-component-analysis-1bdc77462831


<b> Atividade </b>

14) Gere a visualização dos embeddings anteriores utilizando o PCA para reduzir a dimensionalidade.

Exemplo do PCA:

```python
#uso de PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(array_embeddings)
```

In [ ]:
pca = PCA(n_components=2)
pca_result = pca.fit_transform(array_embeddings)

In [ ]:
plot_embedding_2d(pca_result, all_words, words_seed)

#### TSNE
<b> Uma técnica mais recente que captura estrutura não linear é o t-SNE, que significa distribuição estocástica de embedding viziznhos em t ( t-distributed Stochastic Neighbor Embedding).
É uma ótima técnica para capturar a estrutura não linear em dados de alta dimensão(pelo menos em nível local). Isto é, dois pontos que são próximos no espaço de alta dimensão a probabilidade de estarem próximos em uma dimensão baixa é alta. </b>

Mais informações: https://medium.com/@garora039/dimensionality-reduction-using-t-sne-effectively-cabb2cd519b

<b> Atividade </b>

15) Gere a visualização dos embeddings anteriores utilizando o TSNE para reduzir a dimensionalidade.

Exemplo do TSNE:

```python
#uso de TSNE
tsne = TSNE(n_components=2, random_state=0)
tsne_result =  tsne.fit_transform(array_embeddings)
```

In [ ]:
tsne = TSNE(n_components=2, random_state=0,perplexity=4)
tsne_result =  tsne.fit_transform(array_embeddings)

In [ ]:
plot_embedding_2d(tsne_result, all_words, words_seed)

# Similaridade de Documentos

Para realizar a similaridade entre documentos, utilize as frases abaixo:

In [ ]:
frase1 = "Excelente produto chegou antes do prazo indico e recomendo produto bom pois já testei e foi mais que aprovado"
frase2 = "SUPER RECOMENDO, PREÇO, QUALIDADE #BRASTEMP, EFICIÊNCIA NA ENTREGA, E FACILIDADE DE PAGAMENTO. MUITO BOM!!!"
frase3 = "A tampa do fogão veio com problemas com o pino de encaixe solto e precisa de reparos"
frase4 = "Fogão ótimo!"

## Similaridade de Jaccard
Divisão do número de tokens em comuns nas duas frases pelo tamanho do vocabulario das duas frases.
<b> Atividade </b>

16) Faça um método que calcule a similaridade de Jaccard e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4

In [ ]:
def jaccard_sim(frase1, frase2):
  ph1_tokens = pre_processamento_texto(frase1)
  ph2_tokens = pre_processamento_texto(frase2)
  s1 = set(ph1_tokens)
  s2 = set(ph2_tokens)
  return len(s1.intersection(s2)) / len(s1.union(s2))

## Distância e similaridade cosseno

Aqui iremos calcular a distância e a similaridade utilizando embedding

- Embedding

In [ ]:
def embedding_da_frase(tokens_frase):
  return np.mean(np.array([word_vectors[token] for token in tokens_frase if token in word_vectors.key_to_index]), axis=0)

In [ ]:
frase1_emb = embedding_da_frase(frase1_pre)
frase2_emb = embedding_da_frase(frase2_pre)

In [ ]:
distance.cosine(frase1_emb, frase2_emb)

In [ ]:
cosine_similarity(frase1_emb.reshape((1, 100)), frase2_emb.reshape((1, 100)))

In [ ]:
distance.euclidean(frase1_emb, frase2_emb)